## 4.3 The Traditional NLP Approach

### dspy module

In [12]:
import dspy
from sentence_transformers import SentenceTransformer
from transformers import pipeline
lm = dspy.LM('xai/grok-3-mini', max_tokens=100000, temperature=0.1, top_p=0.9)
dspy.configure(lm=lm)

In [2]:
class RAG(dspy.Module):
    def __init__(self, make_search):
        self.make_search = make_search
        self.respond = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

    def forward(self, topic, question, literal, pragmatic):
        search = self.make_search(topic, literal, pragmatic)
        context = " ".join(search(question))
        output = self.respond(context=context, question=question)
        return dspy.Prediction(response=output['answer'])

### test set as list of `dspy.Example`

In [3]:
import json
import os
def get_conversations_set(filename, dataset_dir="../PragmatiCQA/data"):
    conversations = []
    with open(os.path.join(dataset_dir, filename), 'r') as f:
        for line in f:
            conversations.append(json.loads(line))
    return [dspy.Example(topic=d['community'],
                        question=d['qas'][0]['q'], 
                        response=d['qas'][0]['a'],
                        literal=[text['text'] for text in d['qas'][0]['a_meta']['literal_obj']],
                        pragmatic=[text['text'] for text in d['qas'][0]['a_meta']['pragmatic_obj']]).with_inputs('question', 'topic', 'literal', 'pragmatic') for d in conversations]

pcqa_test = get_conversations_set("test.jsonl")

### Evaluation of a model based on example set

In [40]:
from dspy.evaluate import SemanticF1, Evaluate
import pickle
import os
def evaluate(model, example_set, save_path):
    metric = SemanticF1(decompositional=True)

    evaluator = Evaluate(devset=example_set, metric=metric, num_threads=8, display_progress=True, display_table=True, provide_feedback=True)
    results = None
    if os.path.exists(save_path):
        with open(save_path, "rb") as f:
            results = pickle.load(f)
    else:
        results = evaluator(model)
        with open(save_path, "wb") as f:
            pickle.dump(results, f)

    return results

### Getting-Context Methods

In [5]:
# Traverse a directory and read html files - extract text from the html files
from bs4 import BeautifulSoup
import os
def read_html_files(dir_name, directory="../PragmatiCQA-sources"):
    texts = []
    for filename in os.listdir(os.path.join(directory, dir_name)):
        if filename.endswith(".html"):
            with open(os.path.join(directory, dir_name, filename), 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
                texts.append(soup.get_text())
    return texts

model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")
embedder = dspy.Embedder(model.encode)
retriever_dict = {}
def make_search(topic=None, literal=None, pragmatic=None, k=5):
    if topic is None:
        raise ValueError("Topic must be provided for retrieval.")
    corpus = read_html_files(topic)
    max_characters = 10000 
    search = retriever_dict.setdefault((topic, k), dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=k, brute_force_threshold=max_characters))
    return lambda q: search(q).passages


def get_literal(topic=None, literal=None, pragmatic=None):
    if literal is None:
        raise ValueError("Literal must be provided for retrieval.")
    return lambda q: literal

def get_pragmatic(topic=None, literal=None, pragmatic=None):
    if pragmatic is None:
        raise ValueError("Pragmatic must be provided for retrieval.")
    return lambda q: pragmatic

In [6]:
d = {}
print(f"get: {d.get('cat', 0)}")
print(d)


get: 0
{}


### Retrieved Answer

In [9]:
evaluate(RAG(make_search), pcqa_test, "eval_results_distilbert.pkl")

Device set to use cpu


Average Metric: 22.34 / 213 (10.5%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 213/213 [2:09:47<00:00, 36.56s/it]

2025/08/24 23:21:18 INFO dspy.evaluate.evaluate: Average Metric: 22.341969801856195 / 213 (10.5%)


,topic,question,example_response,literal,pragmatic,pred_response,SemanticF1
0,The Legend of Zelda,What year did the Legend of Zelda come out?,The Legend of Zelda came out as early as 1986 for the Famicom in J...,"['FDS release February 21, 1986\n', 'The Legend of Zelda is the fi...","['It came out as early as 1986 for the Famicom in Japan, and was l...",1986,✔️ [0.500]
1,The Legend of Zelda,What console is The Legend of Zelda designed for?,The Legend of Zelda was originally released in 1986 for the Famico...,"[' It came out as early as 1986 for the Famicom in Japan, and was ...","['The Legend of Zelda was commercially successful, selling a milli...",Famicom\n Disk System,✔️ [0.286]
2,The Legend of Zelda,when did the legend of zelda last until?,The Legend of Zelda is the first installment in the Zelda franchis...,['The Legend of Zelda is the first installment in the Zelda franch...,"[Nintendo Switch Online, April 23, 2019]","June 19, 2011",
3,The Legend of Zelda,When was the Legend of Zelda released?,"The Legend of Zelda was released on August 22nd, 1987, for the Nin...","[August 22, 1987]","['It came out as early as 1986 for the Famicom in Japan, and was l...",one hundred years before the events of\n \n Breath of the Wild,
4,The Legend of Zelda,What kind of game is The Legend of Zelda?,"The Legend of Zelda is one that includes roleplaying, action, adve...",[The Legend of Zelda is the first installment of the Zelda series....,"[one that includes roleplaying, action, adventure, and puzzle/logi...",multiplayer,
...,...,...,...,...,...,...,...
208,Baseball,How many times have the new york yankees been to the world series?,"48 times, which is a record actually. Indeed, They have won the wo...",[ 48],"[winning 27 World Series titles, Their 27 titles makes them the mo...",twelve,
209,Baseball,Are the new york yankees based in new york?,"Yes, the New York Yankees now play in New York City but were found...","[The New York Yankees are a Major League Baseball team, based in t...","[The club was founded in Baltimore , Maryland in 1901, and moved t...",New York Yankees\n \n\n\n\n\n One East 161st Street\n ...,✔️ [0.500]
210,Baseball,What sports league does the new York Yankees play in?,The Yankees play in Major League Baseball in the American League a...,[Major League Baseball],"[American League, They are also the only team represented in the B...",Major League Baseball,✔️ [0.500]
211,Baseball,who is the all time leader inh rbi's for the yankees?,Lou Gehrig is the all time leader in runs batted in for the Yankee...,[Lou Gehrig],"['Gehrig finished his career with a .340 AVG, 1,995 RBI', ""During ...",Hank Aaron,


10.49

### Literal Answer

In [14]:
evaluate(RAG(get_literal), pcqa_test, "eval_results_literal.pkl")

Device set to use cpu


Average Metric: 86.80 / 213 (40.8%): 100%|████████████████████████████████████████████| 213/213 [34:46<00:00,  9.80s/it]

2025/08/25 00:01:53 INFO dspy.evaluate.evaluate: Average Metric: 86.79994056425019 / 213 (40.8%)


,topic,question,example_response,literal,pragmatic,pred_response,SemanticF1
0,The Legend of Zelda,What year did the Legend of Zelda come out?,The Legend of Zelda came out as early as 1986 for the Famicom in J...,"['FDS release February 21, 1986\n', 'The Legend of Zelda is the fi...","['It came out as early as 1986 for the Famicom in Japan, and was l...",1986,✔️ [0.500]
1,The Legend of Zelda,What console is The Legend of Zelda designed for?,The Legend of Zelda was originally released in 1986 for the Famico...,"[' It came out as early as 1986 for the Famicom in Japan, and was ...","['The Legend of Zelda was commercially successful, selling a milli...",Famicom,✔️ [0.250]
2,The Legend of Zelda,when did the legend of zelda last until?,The Legend of Zelda is the first installment in the Zelda franchis...,['The Legend of Zelda is the first installment in the Zelda franch...,"[Nintendo Switch Online, April 23, 2019]",first installment in the Zelda franchise,✔️ [0.333]
3,The Legend of Zelda,When was the Legend of Zelda released?,"The Legend of Zelda was released on August 22nd, 1987, for the Nin...","[August 22, 1987]","['It came out as early as 1986 for the Famicom in Japan, and was l...","August 22, 1987",✔️ [0.667]
4,The Legend of Zelda,What kind of game is The Legend of Zelda?,"The Legend of Zelda is one that includes roleplaying, action, adve...",[The Legend of Zelda is the first installment of the Zelda series....,"[one that includes roleplaying, action, adventure, and puzzle/logi...",Zelda,✔️ [0.286]
...,...,...,...,...,...,...,...
208,Baseball,How many times have the new york yankees been to the world series?,"48 times, which is a record actually. Indeed, They have won the wo...",[ 48],"[winning 27 World Series titles, Their 27 titles makes them the mo...",48,✔️ [0.400]
209,Baseball,Are the new york yankees based in new york?,"Yes, the New York Yankees now play in New York City but were found...","[The New York Yankees are a Major League Baseball team, based in t...","[The club was founded in Baltimore , Maryland in 1901, and moved t...",The New York Yankees,✔️ [0.500]
210,Baseball,What sports league does the new York Yankees play in?,The Yankees play in Major League Baseball in the American League a...,[Major League Baseball],"[American League, They are also the only team represented in the B...",Major League Baseball,✔️ [0.500]
211,Baseball,who is the all time leader inh rbi's for the yankees?,Lou Gehrig is the all time leader in runs batted in for the Yankee...,[Lou Gehrig],"['Gehrig finished his career with a .340 AVG, 1,995 RBI', ""During ...",Lou Gehrig,✔️ [0.286]


40.75

### Pragmatic Answer

In [15]:
evaluate(RAG(get_pragmatic), pcqa_test, "eval_results_pragmatic.pkl")

Device set to use cpu


Average Metric: 74.43 / 213 (34.9%): 100%|████████████████████████████████████████████| 213/213 [31:15<00:00,  8.81s/it]

2025/08/25 00:33:10 INFO dspy.evaluate.evaluate: Average Metric: 74.42974948540159 / 213 (34.9%)


,topic,question,example_response,literal,pragmatic,pred_response,SemanticF1
0,The Legend of Zelda,What year did the Legend of Zelda come out?,The Legend of Zelda came out as early as 1986 for the Famicom in J...,"['FDS release February 21, 1986\n', 'The Legend of Zelda is the fi...","['It came out as early as 1986 for the Famicom in Japan, and was l...",1986,✔️ [0.500]
1,The Legend of Zelda,What console is The Legend of Zelda designed for?,The Legend of Zelda was originally released in 1986 for the Famico...,"[' It came out as early as 1986 for the Famicom in Japan, and was ...","['The Legend of Zelda was commercially successful, selling a milli...",Nintendo Entertainment System,✔️ [0.333]
2,The Legend of Zelda,when did the legend of zelda last until?,The Legend of Zelda is the first installment in the Zelda franchis...,['The Legend of Zelda is the first installment in the Zelda franch...,"[Nintendo Switch Online, April 23, 2019]","April 23, 2019",✔️ [0.400]
3,The Legend of Zelda,When was the Legend of Zelda released?,"The Legend of Zelda was released on August 22nd, 1987, for the Nin...","[August 22, 1987]","['It came out as early as 1986 for the Famicom in Japan, and was l...",1987,✔️ [0.667]
4,The Legend of Zelda,What kind of game is The Legend of Zelda?,"The Legend of Zelda is one that includes roleplaying, action, adve...",[The Legend of Zelda is the first installment of the Zelda series....,"[one that includes roleplaying, action, adventure, and puzzle/logi...",roleplaying,✔️ [0.286]
...,...,...,...,...,...,...,...
208,Baseball,How many times have the new york yankees been to the world series?,"48 times, which is a record actually. Indeed, They have won the wo...",[ 48],"[winning 27 World Series titles, Their 27 titles makes them the mo...",27,✔️ [0.400]
209,Baseball,Are the new york yankees based in new york?,"Yes, the New York Yankees now play in New York City but were found...","[The New York Yankees are a Major League Baseball team, based in t...","[The club was founded in Baltimore , Maryland in 1901, and moved t...",New York in 1903.,✔️ [0.400]
210,Baseball,What sports league does the new York Yankees play in?,The Yankees play in Major League Baseball in the American League a...,[Major League Baseball],"[American League, They are also the only team represented in the B...",American League,✔️ [0.500]
211,Baseball,who is the all time leader inh rbi's for the yankees?,Lou Gehrig is the all time leader in runs batted in for the Yankee...,[Lou Gehrig],"['Gehrig finished his career with a .340 AVG, 1,995 RBI', ""During ...",Gehrig,✔️ [0.286]


34.94

## 4.4 Multi Step LLM 

### Signatures

In [13]:
import dspy

# --- Intermediates ---
class SummarizeGoal(dspy.Signature):
    """Summarize the student's long-term goal or interest from the dialogue history to guide cooperative answering."""
    history = dspy.InputField(desc="list of (question, answer) strings from prior turns")
    goal_summary = dspy.OutputField(desc="concise 1-3 sentence summary of the student's goal, preferences, constraints")

class InferNeed(dspy.Signature):
    """Infer the pragmatic, cooperative need underlying the current question, using history and retrieved context."""
    history = dspy.InputField()
    question = dspy.InputField()
    context = dspy.InputField()
    need = dspy.OutputField(desc="brief description of what the student is really trying to accomplish right now")

class GenerateCoopQuery(dspy.Signature):
    """Generate a cooperative follow-up query that would retrieve missing or clarifying info from the corpus."""
    history = dspy.InputField()
    question = dspy.InputField()
    need = dspy.InputField()
    coop_query = dspy.OutputField(desc="a single, highly targeted search query or question to re-query the corpus")

# --- Final answering step with CoT ---
class CooperativeAnswer(dspy.Signature):
    """Produce a cooperative, step-by-step answer grounded in all context, surfacing assumptions and next steps."""
    history = dspy.InputField()
    question = dspy.InputField()
    goal_summary = dspy.InputField()
    need = dspy.InputField()
    context = dspy.InputField(desc="initial retrieved context")
    extra_context = dspy.InputField(desc="additional context retrieved via the cooperative query")
    answer = dspy.OutputField(desc="final cooperative answer with citations if helpful")


### Module

In [14]:
class CooperativeRAG(dspy.Module):
    def __init__(self, make_search, k_initial=6, k_extra=4):
        super().__init__()
        self.make_search = make_search
        self.k_initial = k_initial
        self.k_extra = k_extra

        # Submodules (you can swap these for Instruct/Refine variants if you like)
        self.summarize_goal = dspy.Predict(SummarizeGoal)
        self.infer_need = dspy.Predict(InferNeed)
        self.generate_coop_query = dspy.Predict(GenerateCoopQuery)
        # Chain-of-Thought final step
        self.answerer = dspy.ChainOfThought(CooperativeAnswer)

    def _format_history(self, history_pairs):
        # history_pairs: list[{"question": str, "answer": str}]
        lines = []
        for i, t in enumerate(history_pairs, 1):
            q = t.get("question", "").strip()
            a = t.get("answer", "").strip()
            lines.append(f"Turn {i} - Q: {q}\nTurn {i} - A: {a}")
        return "\n".join(lines) if lines else "No prior turns."

    def _retrieve_initial(self, topic, question):
        search = self.make_search(topic=topic, k=self.k_initial)
        docs = search(question)
        return "\n\n".join(docs)

    def _retrieve_extra(self, topic, coop_query):
        if not coop_query or coop_query.strip() == "":
            return ""
        
        search = self.make_search(topic=topic, k=self.k_extra)
        docs = search(coop_query)
        return "\n\n".join(docs)

    def forward(self, topic, question, history):
        # 1) Format inputs
        history_text = self._format_history(history)

        # 2) Initial retrieval (if you prefer to pipe in context externally, you can)
        initial_context = self._retrieve_initial(topic, question)

        # 3) Intermediates
        goal = self.summarize_goal(history=history_text).goal_summary
        need = self.infer_need(history=history_text, question=question, context=initial_context).need
        coop_query = self.generate_coop_query(history=history_text, question=question, need=need).coop_query

        # 4) Extra retrieval using cooperative query
        extra_context = self._retrieve_extra(topic, coop_query)

        # 5) Final, cooperative answer
        result = self.answerer(
            history=history_text,
            question=question,
            goal_summary=goal,
            need=need,
            context=initial_context,
            extra_context=extra_context
        )

        return dspy.Prediction(
            goal_summary=goal,
            need=need,
            coop_query=coop_query,
            response=result.answer
        )

### 4.4.1 First questions

In [23]:

def get_first_qs(filename, dataset_dir="../PragmatiCQA/data"):
    conversations = []
    with open(os.path.join(dataset_dir, filename), 'r') as f:
        for line in f:
            conversations.append(json.loads(line))
    return [dspy.Example(topic=d['community'],
                        question=d['qas'][0]['q'], 
                        response=d['qas'][0]['a'],
                        history=[]).with_inputs('question', 'topic', 'history') for d in conversations]


first_qs = get_first_qs("test.jsonl")[:20]

In [24]:
evaluate(CooperativeRAG(make_search), first_qs, "eval_results_coop_1Q.pkl")

Average Metric: 4.92 / 12 (41.0%):  60%|███████████████████████████                  | 12/20 [1:06:53<49:23, 370.43s/it]

2025/08/27 14:15:19 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 5.69 / 14 (40.6%):  70%|██████████████████████████████             | 14/20 [1:36:39<1:00:06, 601.14s/it]

2025/08/27 14:50:10 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 8.95 / 20 (44.7%): 100%|█████████████████████████████████████████████| 20/20 [2:26:35<00:00, 439.80s/it]

2025/08/27 15:14:31 INFO dspy.evaluate.evaluate: Average Metric: 8.949793263706342 / 20 (44.7%)


,topic,question,example_response,history,goal_summary,need,coop_query,pred_response,SemanticF1
0,The Legend of Zelda,What year did the Legend of Zelda come out?,The Legend of Zelda came out as early as 1986 for the Famicom in J...,[],"Based on the available history, no specific goals, preferences, or...",The student is seeking to obtain the original release year of The ...,What is the original release year of The Legend of Zelda video gam...,"Based on the information from the context, The Legend of Zelda was...",✔️ [0.571]
1,The Legend of Zelda,What console is The Legend of Zelda designed for?,The Legend of Zelda was originally released in 1986 for the Famico...,[],"Based on the available history, no specific goals, preferences, or...",The student is seeking information on the original console for The...,What was the original console for the game The Legend of Zelda?,"The original game, *The Legend of Zelda*, was designed for the Fam...",✔️ [0.364]
2,The Legend of Zelda,when did the legend of zelda last until?,The Legend of Zelda is the first installment in the Zelda franchis...,[],"Based on the available history, no specific goals, preferences, or...",The student is seeking to understand the timeline or historical sp...,What is the release date of the most recent game in The Legend of ...,Thank you for your question about The Legend of Zelda series! It s...,✔️ [0.571]
3,The Legend of Zelda,When was the Legend of Zelda released?,"The Legend of Zelda was released on August 22nd, 1987, for the Nin...",[],"Based on the available history, no specific goals, preferences, or...",The student is seeking the release date of the original Legend of ...,What is the exact release date of the original Legend of Zelda vid...,"The original Legend of Zelda game was released on February 21, 198...",✔️ [1.000]
4,The Legend of Zelda,What kind of game is The Legend of Zelda?,"The Legend of Zelda is one that includes roleplaying, action, adve...",[],"Based on the available history, no specific goals, preferences, or...",The student is seeking a description of the genre and key features...,"What is the genre, key gameplay features, and historical significa...",The Legend of Zelda is an action-adventure game developed by Ninte...,✔️ [0.417]
5,The Legend of Zelda,What year was this game release?,Great questions its first release was in 1986 but I guess the main...,[],"Based on the available history, no specific goals, preferences, or...",The student wants to determine the release year of The Legend of Z...,What is the release year of The Legend of Zelda: Collector's Edition?,The Legend of Zelda: Collector's Edition was primarily released in...,
6,The Legend of Zelda,where is zelda from?,Princess Zelda herself is from a little kingdom in the land of Hyr...,[],"Based on the available history, no specific goals, preferences, or...",The student is trying to learn about the fictional origins or back...,What is the fictional origin or homeland of Princess Zelda in The ...,Princess Zelda is a fictional character from The Legend of Zelda s...,✔️ [0.143]
7,The Legend of Zelda,Tell me when did the legend of Zelda come out?,The Legend of Zelda is the first installment of the Zelda series. ...,[],"Based on the available history, no specific goals, preferences, or...",The student is seeking the original release date of the video game...,"What is the original release date of the video game ""The Legend of...","Based on the information from the context, the original ""The Legen...",✔️ [0.727]
8,The Legend of Zelda,What system did the game first come out in?,It was first released on NES (the Nintendo Entertainment System) i...,[],"Based on the available history, no specific goals, preferences, or...",The student is seeking to identify the original gaming system for ...,What was the original gaming console or platform for the first rel...,The Legend of Zelda first came out on the Famicom Disk System in J...,
9,The Legend of Zelda,Who is Zelda?,Princess Zelda is the ep

44.75

### 4.4.2 All questions

In [38]:
def get_qs_with_history(filename, dataset_dir="../PragmatiCQA/data"):
    conversations = []
    with open(os.path.join(dataset_dir, filename), 'r') as f:
        for line in f:
            conversations.append(json.loads(line))

    examples = []
    return [dspy.Example(topic=d['community'],
                        question=pair['q'], 
                        response=pair['a'],
                        history=[{"question": past['q'], "answer": past['a']} for past in d['qas'][:i]]
    ).with_inputs('question', 'topic', 'history') for d in conversations for i, pair in enumerate(d['qas'])]


all_qs = get_qs_with_history("test.jsonl")

In [41]:
evaluate(CooperativeRAG(make_search), all_qs[:20], "eval_results_coop_allQ.pkl")

Average Metric: 9.53 / 19 (50.2%): 100%|███████████████████████████████████████████████| 20/20 [24:26<00:00, 137.61s/it]

2025/08/27 16:43:20 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 9.53 / 19 (50.2%): : 22it [35:56, 257.99s/it]                                                           

2025/08/27 16:54:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/27 17:14:31 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/27 17:14:31 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 9.97 / 20 (49.9%): : 23it [1:48:25, 282.85s/it] 

2025/08/27 18:06:22 INFO dspy.evaluate.evaluate: Average Metric: 9.97403173960517 / 20 (49.9%)


2025/08/27 18:06:22 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


,topic,question,example_response,history,goal_summary,need,coop_query,pred_response,SemanticF1
0,The Legend of Zelda,What year did the Legend of Zelda come out?,The Legend of Zelda came out as early as 1986 for the Famicom in J...,[],"Based on the available history, no specific goals, preferences, or...",The student is seeking to obtain the original release year of The ...,What is the original release year of The Legend of Zelda video gam...,"Based on the information from the context, The Legend of Zelda was...",✔️ [0.571]
1,The Legend of Zelda,Yes who are the main characters in the story?,"The series centers on Link, the playable character and protagonist...","[{'question': 'What year did the Legend of Zelda come out?', 'answ...",The student is inquiring about basic historical facts regarding Th...,The student is seeking information on the main characters in The L...,What are the main characters in the original story of The Legend o...,Thank you for your follow-up question! It's great to see your inte...,✔️ [0.686]
2,The Legend of Zelda,What settings have appeared in this and other games in the series?,"The first setting is the ""Dungeon Exploration"" barring Link 's pro...","[{'question': 'What year did the Legend of Zelda come out?', 'answ...",The student is interested in learning about the history and key el...,The student is seeking to explore and understand the various locat...,What are the key locations and environments featured across the va...,Thank you for your question about the settings in The Legend of Ze...,✔️ [0.200]
3,The Legend of Zelda,Yes I would like to learn more about the overworld.,"The Overworld Exploration setting a non-linearity, the ability to ...","[{'question': 'What year did the Legend of Zelda come out?', 'answ...","The student is interested in exploring the history, characters, an...",The student wants to expand their understanding of the Overworld i...,What are the key Overworld locations in The Legend of Zelda series...,Great to see your interest in the Overworld continuing from our la...,✔️ [0.429]
4,The Legend of Zelda,Yes I would like to learn more about the second quest.,The Second Quest... after completing the game a player has access ...,"[{'question': 'What year did the Legend of Zelda come out?', 'answ...",The student is interested in exploring the details of the Legend o...,The student is seeking to expand their knowledge of The Legend of ...,"What are the key features, differences, and gameplay changes in th...",Thank you for your continued interest in The Legend of Zelda serie...,✔️ [0.750]
5,The Legend of Zelda,Yeah that sounds good. I would like to know who is credited for cr...,Game Information...Shortly before starting development for Super M...,"[{'question': 'What year did the Legend of Zelda come out?', 'answ...",The student's primary goal is to gain detailed knowledge about the...,The student is seeking to expand their knowledge of the Legend of ...,Who is credited as the primary creator and originator of the idea ...,Based on the historical details from the development of The Legend...,✔️ [0.545]
6,The Legend of Zelda,What console is The Legend of Zelda designed for?,The Legend of Zelda was originally released in 1986 for the Famico...,[],"Based on the available history, no specific goals, preferences, or...",The student is seeking information on the original console for The...,What was the original console for the game The Legend of Zelda?,"The original game, *The Legend of Zelda*, was designed for the Fam...",✔️ [0.364]
7,The Legend of Zelda,What is a NES?,The NES - or Nintendo Entertainment System - was a game console la...,"[{'question': 'What console is The Legend of Zelda designed for?',...",The student is inquiring about the consoles for The Legend of Zeld...,The student is seeking clarification on what the NES (Nintendo Ent...,What is the Nintendo Entertainment System (NES) and its role as a ...,"The NES, short for Nintendo Entertainment System, is a home video ..."

49.87